<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Ecommerce-Churn-Assignment" data-toc-modified-id="Ecommerce-Churn-Assignment-1">Ecommerce Churn Assignment</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Data-description" data-toc-modified-id="Data-description-1.0.1">Data description</a></span></li><li><span><a href="#Initialising-the-SparkSession" data-toc-modified-id="Initialising-the-SparkSession-1.0.2">Initialising the SparkSession</a></span></li></ul></li><li><span><a href="#Task-3:-Model-Selection" data-toc-modified-id="Task-3:-Model-Selection-1.1">Task 3: Model Selection</a></span><ul class="toc-item"><li><span><a href="#Model-2:-Decision-Trees" data-toc-modified-id="Model-2:-Decision-Trees-1.1.1">Model 2: Decision Trees</a></span><ul class="toc-item"><li><span><a href="#Model-Fitting" data-toc-modified-id="Model-Fitting-1.1.1.1">Model Fitting</a></span></li></ul></li></ul></li><li><span><a href="#Accuracy" data-toc-modified-id="Accuracy-1.2">Accuracy</a></span></li></ul></li><li><span><a href="#Precision" data-toc-modified-id="Precision-2">Precision</a></span></li><li><span><a href="#Recall" data-toc-modified-id="Recall-3">Recall</a></span></li><li><span><a href="#F1-score" data-toc-modified-id="F1-score-4">F1 score</a></span></li><li><span><a href="#Best-model-param" data-toc-modified-id="Best-model-param-5">Best model param</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Model-Analysis" data-toc-modified-id="Model-Analysis-5.0.0.1">Model Analysis</a></span></li><li><span><a href="#Summary-of-the-best-Decision-Tree-model" data-toc-modified-id="Summary-of-the-best-Decision-Tree-model-5.0.0.2">Summary of the best Decision Tree model</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Ecommerce Churn Assignment



The aim of the assignment is to build a model that predicts whether a person purchases an item after it has been added to the cart or not. Being a classification problem, you are expected to use your understanding of all the three models covered till now. You must select the most robust model and provide a solution that predicts the churn in the most suitable manner. 

For this assignment, you are provided the data associated with an e-commerce company for the month of October 2019. Your task is to first analyse the data, and then perform multiple steps towards the model building process.

The broad tasks are:
- Data Exploration
- Feature Engineering
- Model Selection
- Model Inference

### Data description

The dataset stores the information of a customer session on the e-commerce platform. It records the activity and the associated parameters with it.

- **event_time**: Date and time when user accesses the platform
- **event_type**: Action performed by the customer
            - View
            - Cart
            - Purchase
            - Remove from cart
- **product_id**: Unique number to identify the product in the event
- **category_id**: Unique number to identify the category of the product
- **category_code**: Stores primary and secondary categories of the product
- **brand**: Brand associated with the product
- **price**: Price of the product
- **user_id**: Unique ID for a customer
- **user_session**: Session ID for a user


### Initialising the SparkSession

The dataset provided is 5 GBs in size. Therefore, it is expected that you increase the driver memory to a greater number. The m4.xlarge instance holds 16 GB memory and we are allocating 14 GB to the driver.

In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.driver.memory": "14000M"}
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1595331360318_0003,pyspark,idle,Link,Link,
2,application_1595331360318_0004,pyspark,idle,Link,Link,


In [2]:
# init spark session
spark.sparkContext.getConf().get('spark.driver.memory')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1595331360318_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'14000M'

In [3]:
# What libs are available
spark.sparkContext.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.9.0  
boto                       2.49.0 
jmespath                   0.9.5  
lxml                       4.5.0  
mysqlclient                1.4.2  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.16.5 
pip                        9.0.1  
py-dateutil                2.2    
python37-sagemaker-pyspark 1.3.0  
pytz                       2019.3 
PyYAML                     5.3.1  
setuptools                 28.8.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.29.0 
windmill                   1.6

In [4]:
# installing required libraries
spark.sparkContext.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
spark.sparkContext.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository
spark.sparkContext.install_pypi_package("numpy==1.16.5")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/1c/15/3fea1bfb7e5b77b7cca9c6010a9cabc58ea125385345ecb6f5832eb8b49a/matplotlib-3.3.0-1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/f2/6722dd0c22e3a143ac792ccb2424924ac72af4adea756b1165b4cad50da7/Pillow-7.2.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using

In [5]:
# loading the dataset from the parquet file
df = spark.read.parquet("s3://shiva-test-bucket-2/NOTEBOOK_1_V2.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Check if the dataframe is correctly loaded
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+-----------+-----------+----------+------------+-------------+--------------+-------------------------+-------------------------+------------------+----------------+--------------+----------------+---------------+-----------------+---------------+----------------+---------------+-----------------+-----+--------------------+
|  category_1| category_2|avg_expense|time_of_day|top_brands|is_purchased|day_of_week_d|product_view_d|event_per_session_count_d|sec_category_view_count_d|distinct_session_d|category_1_index|category_1_vec|category_2_index| category_2_vec|time_of_day_index|time_of_day_vec|top_brands_index| top_brands_vec|day_of_week_d_vec|label|            features|
+------------+-----------+-----------+-----------+----------+------------+-------------+--------------+-------------------------+-------------------------+------------------+----------------+--------------+----------------+---------------+-----------------+---------------+----------------+------

In [7]:
df = df.drop('is_purchased')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# exploring the dataframe - schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- category_1: string (nullable = true)
 |-- category_2: string (nullable = true)
 |-- avg_expense: double (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- top_brands: string (nullable = true)
 |-- day_of_week_d: double (nullable = true)
 |-- product_view_d: double (nullable = true)
 |-- event_per_session_count_d: double (nullable = true)
 |-- sec_category_view_count_d: double (nullable = true)
 |-- distinct_session_d: double (nullable = true)
 |-- category_1_index: double (nullable = true)
 |-- category_1_vec: vector (nullable = true)
 |-- category_2_index: double (nullable = true)
 |-- category_2_vec: vector (nullable = true)
 |-- time_of_day_index: double (nullable = true)
 |-- time_of_day_vec: vector (nullable = true)
 |-- top_brands_index: double (nullable = true)
 |-- top_brands_vec: vector (nullable = true)
 |-- day_of_week_d_vec: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

In [9]:
# Number of rows and columns in the dataset
df.count(), len(df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(10900919, 21)

<hr>

## Task 3: Model Selection
3 models for classification:	
- Logistic Regression
- Decision Tree
- Random Forest

### Model 2: Decision Trees

In [10]:
# Splitting the data into train and test (Remember you are expected to compare the model later)
df_train, df_test = df.randomSplit([0.7, 0.3], seed = 2020)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Number of rows in train and test data
df_train.count(), df_test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(7631817, 3269102)

#### Model Fitting

In [12]:
# Building the model with hyperparameter tuning
# Create ParamGrid for Cross Validation
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidatorModel, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Instantiating Decision tree
dtree = DecisionTreeClassifier(labelCol = 'label', featuresCol = 'features', maxDepth = 2)

# Create grid  for cross validation
grid = (ParamGridBuilder()\
.addGrid(dtree.maxDepth, [5, 10])\
.addGrid(dtree.maxBins, [60, 100])\
.build())

# evaluate the model
#dtree_evaluation = BinaryClassificationEvaluator()
dtree_evaluator = MulticlassClassificationEvaluator(labelCol = 'label',
                                             predictionCol = 'prediction',
                                             metricName = 'accuracy')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Run cross-validation steps
dtree_cv = CrossValidator(estimator = dtree,
                         estimatorParamMaps = grid,
                         evaluator = dtree_evaluator,
                          parallelism = 2,
                         numFolds = 3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Fitting the models on transformed df
dtree_cv_model = dtree_cv.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-13:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'

Exception in thread cell_monitor-11:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _boots

In [15]:
# use model to transform test data
predictions = dtree_cv_model.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-14:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [16]:
predictions.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-15:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-----------+----------+-----------+-----------+----------+-------------+--------------+-------------------------+-------------------------+------------------+----------------+--------------+----------------+---------------+-----------------+---------------+----------------+--------------+-----------------+-----+--------------------+---------------+-----------+----------+
| category_1|category_2|avg_expense|time_of_day|top_brands|day_of_week_d|product_view_d|event_per_session_count_d|sec_category_view_count_d|distinct_session_d|category_1_index|category_1_vec|category_2_index| category_2_vec|time_of_day_index|time_of_day_vec|top_brands_index|top_brands_vec|day_of_week_d_vec|label|            features|  rawPrediction|probability|prediction|
+-----------+----------+-----------+-----------+----------+-------------+--------------+-------------------------+-------------------------+------------------+----------------+--------------+----------------+---------------+-----------------+--------

In [17]:
# Decision Trees prediction
predictions.write.option("header", "true").parquet("s3://shiva-test-bucket-2/decision_tree_predictions.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-16:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



## Accuracy

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                  predictionCol="prediction",
                                  metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# accuracy
print("Model Accuracy: {}".format(accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-17:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



Model Accuracy: 0.9863739338815369

In [19]:
#import the libraries required to print the confusion matrix
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
import pyspark.sql.functions as F


#Place the predictions and labels together
preds_and_labels = predictions.select(['prediction','label'])

#convert into RDD
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-18:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [20]:
#print in form of an array
print(metrics.confusionMatrix().toArray())

VBox()

Exception in thread cell_monitor-19:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[3160576.   30808.]
 [  13737.   63981.]]

# Precision

In [21]:
precision = metrics.precision()
# Precision for label 0
precision_0 = metrics.precision(0)
# Precision for label1
precision_1 = metrics.precision(1)

print("Oveerall Precision = {}".format(precision))
print("Negavive class Precision = {}".format(precision_0))
print("Positive class Precision = {}".format(precision_1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-20:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



Oveerall Precision = 0.9863739338815369
Negavive class Precision = 0.9956724494402411
Positive class Precision = 0.6749833841479497

# Recall

In [22]:
recall = metrics.recall()
# Recall for label1
recall_1 = metrics.recall(1)
# Recall for label0
recall_0 = metrics.recall(0)

print("Overall recall = {}".format(recall))
print("Negavive class recall = {}".format(recall_0))
print("Positive class recall = {}".format(recall_1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Overall recall = 0.9863739338815369
Negavive class recall = 0.9903465079727165
Positive class recall = 0.8232455801744769

Exception in thread cell_monitor-21:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



# F1 score

In [23]:
#Calculate f1 score
f1Score = metrics.fMeasure()
f1Score

# F1 score for label 0
f1_score_0 = metrics.fMeasure(0.0, beta = 1.0)
# F1 score for label 1
f1_score_1 = metrics.fMeasure(1.0, beta = 1.0)

### f1 score by label

f1_score_0, f1_score_1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.9930023373716972, 0.7417785944918178)

Exception in thread cell_monitor-22:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



# Best model param

In [24]:
# Best model from the results of cross-validation
print(" Max number of Bins : {}".format(dtree_cv_model.bestModel._java_obj.getMaxBins()))
print(" Max Depth of the tree: {}".format(dtree_cv_model.bestModel._java_obj.getMaxDepth()))

#convert into RDD
#metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Max number of Bins : 100
 Max Depth of the tree: 10

Exception in thread cell_monitor-23:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



#### Model Analysis

Required Steps:
- Fit on test data
- Performance analysis
    - Appropriate Metric with reasoning

The Model is porforming well on the test data, with over Recall and precision at 0.98

The label Specific precision and recall is also calculates

The recall for positive label is 0.82324, this is good 

f1 score for positive class is which is also acceptable 0.7417

#### Summary of the best Decision Tree model